# Projeto Final: Avaliação dos modelos 

Neste notebook serão avaliados os dois modelos propostos para o projeto final da disciplina IA376 de 2020/2.



# Dependências

In [ ]:
!pip install datasets             --quiet
!pip install efficientnet-pytorch --quiet
!pip install transformers==3.5.1  --quiet
!pip install pytorch-lightning    --quiet
!pip install jsonlines            --quiet

In [ ]:
import os
import glob
import json
import spacy
import torch
import datasets
import jsonlines
import collections
import torchvision
import efficientnet_pytorch
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import torch.nn.functional as F


import numpy as np
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, TensorDataset


from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageChops
from random import randrange
from datasets import load_dataset, list_datasets
from torchvision import transforms, utils
from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm.notebook import tqdm
from matplotlib.pyplot import imshow
from efficientnet_pytorch import EfficientNet
from pytorch_lightning.callbacks import ModelCheckpoint

if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu" 
print(dev, torch.cuda.get_device_name(0))
device = torch.device(dev)

cuda:0 Tesla T4


# Métricas

In [ ]:
def normalize_answer(s):
    """Lower text and remove extra whitespace."""

    def white_space_fix(text):
        return ' '.join(text.split())

    def lower(text):
        return text.lower()

    return white_space_fix(lower(s))

def get_tokens(s):
    if not s: return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

# Modelo

In [ ]:
class T5Finetuner(pl.LightningModule):

    def __init__(self, train_dataloader, val_dataloader, test_dataloader, params):
        super(T5Finetuner, self).__init__()

        self.params = params
        
        self._train_dataloader = train_dataloader
        self._val_dataloader   = val_dataloader
        self._test_dataloader  = test_dataloader

        self.encoder = EfficientNet.from_pretrained(model_name='efficientnet-b0', 
                                                    advprop= params['advprop'])
        self.decoder = T5ForConditionalGeneration.from_pretrained('t5-base')

        self.cnn1 = nn.Conv2d(in_channels=112, out_channels=
                              self.decoder.config.d_model, kernel_size=1)

        self.tokenizer = T5Tokenizer.from_pretrained('t5-base')

        self.learning_rate = params['learning_rate']

    def Embeddings(self, images):

        # Shape (N, 112, 16, 16)
        features = model.encoder.extract_endpoints(images)["reduction_4"] 
        features = self.cnn1(features)
        embeddings = features.permute(0, 2, 3, 1).reshape(features.shape[0], -1, 
                                                          self.decoder.config.d_model)

        return embeddings

    def generate(self, embeddings):
        ''' 
        A partir das features extraidas da EfNet, gera o texto com o 
        encoder + decoder do T5.
        '''
        max_length = self.params['max_seq_len']

        # Add start of sequence token
        decoded_ids = torch.full((embeddings.shape[0], 1),
                                 self.decoder.config.decoder_start_token_id,
                                 dtype=torch.long).to(embeddings.device)

        encoder_hidden_states = self.decoder.get_encoder()(inputs_embeds=embeddings)

        for step in range(max_length-1):
            logits = self.decoder(decoder_input_ids=decoded_ids, 
                                  encoder_outputs=encoder_hidden_states)[0]
            next_token_logits = logits[:, -1, :]

            # Greedy decoding
            next_token_id = next_token_logits.argmax(1).unsqueeze(-1)
            
            # Check if output is end of senquence for all batches
            if torch.eq(next_token_id[:, -1], self.tokenizer.eos_token_id).all():
                break

            # Concatenate past ids with new id, keeping batch dimension
            decoded_ids = torch.cat([decoded_ids, next_token_id], dim=-1)

        return decoded_ids

    def forward(self, batch): 
        images, labels, tokens = batch 

        embeddings = self.Embeddings(images)

        if self.training:
            outputs = self.decoder(inputs_embeds=embeddings,
                                   decoder_input_ids=None, 
                                   labels=tokens, return_dict=True)
            return outputs.loss
        
        else:
            return self.generate(embeddings)


    def training_step(self, batch, batch_idx): 
        loss = self(batch)
        self.log('loss', loss, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        pred_tokens = self(batch)
        decoded_pred = [self.tokenizer.decode(tokens) for tokens in pred_tokens]
        return {"pred": decoded_pred, "target": batch[1]}

    def test_step(self, batch, batch_idx):
        pred_tokens = self(batch)
        decoded_pred = [self.tokenizer.decode(tokens) for tokens in pred_tokens]
        return {"pred": decoded_pred, "target": batch[1]}

    def validation_epoch_end(self, outputs):
        trues = sum([list(x['target']) for x in outputs], [])
        preds = sum([list(x['pred']) for x in outputs], [])
        # Exibe um exemplo aleatório do conjunto de validação
        n = randrange(len(trues))
        print(f"\nSample Target: {trues[n]}\nPrediction: {preds[n]}\n")

        f1 = []
        exact = []
        for true, pred in zip(trues, preds):
            f1.append(compute_f1(a_gold=true, a_pred=pred))
            exact.append(compute_exact(a_gold=true, a_pred=pred))
        f1 = np.mean(f1)
        exact = np.mean(exact)

        self.log("val_f1", f1, prog_bar=True)
        self.log("val_exact", exact, prog_bar=True)

    def test_epoch_end(self, outputs):
        # Flatten dos targets e preds para arrays
        trues = sum([list(x['target']) for x in outputs], [])
        preds = sum([list(x['pred']) for x in outputs], [])

        for random in range(5):
            n = randrange(len(trues))
            print(f"\nSample Target: {trues[n]}\nPrediction: {preds[n]}\n")

        f1 = []
        exact = []
        for true, pred in zip(trues, preds):
            f1.append(compute_f1(a_gold=true, a_pred=pred))
            exact.append(compute_exact(a_gold=true, a_pred=pred))
        f1 = np.mean(f1)
        exact = np.mean(exact)

        self.log("test_f1", f1, prog_bar=True)
        self.log("test_exact", exact, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(
            [p for p in self.parameters() if p.requires_grad],
            lr=self.learning_rate, eps=1e-08)

    def train_dataloader(self):
        return self._train_dataloader

    def val_dataloader(self):
        return self._val_dataloader

    def test_dataloader(self):
        return self._test_dataloader

# Parâmetros

In [ ]:
params = {
    'advprop': True,          
    'batch_size': 8,
    'max_seq_len': 128,
    'learning_rate': 5e-4,
    'max_epochs': 70,
    'patience': 25,
    'monitor_variable': 'val_f1',
    'augmentation': 5
}

# Modelo pré-treinado no Synthetic Word Dataset 
## ICDAR SROIE 2019 - Avaliação no conjunto de teste

In [ ]:
resume_from_checkpoint = 'img2text-synteticworddataset--epoch=0-val_exact=0.95-v0.ckpt'

trainer = pl.Trainer(
    gpus=1,
    precision=32, 
    log_gpu_memory=True,
    max_epochs=1,
    check_val_every_n_epoch=1,
    profiler=True,
    callbacks=None,
    progress_bar_refresh_rate=1,
    resume_from_checkpoint=resume_from_checkpoint)

In [ ]:
trainer.test(model)


Sample Target: Company: SANYU STATIONERY SHOP Address: NO. 31G&33G, JALAN SETIA INDAH X ,U13/X 40170 SETIA ALAM
Prediction: Company: MR. D.I.Y. (M) SDN BHD Address: LOT 1851-A & 1851-B, JALAN KPB 6, KAWASAN PERINDUSTRIAN BALAKONG, 43300 SERI KEMBANGAN, SELANGOR


Sample Target: Company: MR. D.I.Y. (M) SDN BHD Address: LOT 1851-A & 1851-B, JALAN KPB 6, KAWASAN PERINDUSTRIAN BALAKONG, 43300 SERI KEMBANGAN, SELANGOR
Prediction: Company: MR. D.I.Y. (M) SDN BHD Address: LOT 1851-A & 1851-B, JALAN KPB 6, KAWASAN PERINDUSTRIAN BALAKONG, 43300 SERI KEMBANGAN, SELANGOR


Sample Target: Company: RESTAURANT JIAWEI JIAWEI HOUSE Address: 13, JLN TASIK UTAMA 8 MEDAN NIAGA DAMAI SG BESI 57000 KL
Prediction: Company: MR. D.I.Y. (M) SDN BHD Address: LOT 1851-A & 1851-B, JALAN KPB 6, KAWASAN PERINDUSTRIAN BALAKONG, 43300 SERI KEMBANGAN, SELANGOR


Sample Target: Company: SANYU STATIONERY SHOP Address: NO. 31G&33G, JALAN SETIA INDAH X ,U13/X 40170 SETIA ALAM
Prediction: Company: MR. D.I.Y. (M) SDN BHD A

[{'test_exact': 0.1407035175879397, 'test_f1': 0.3891333301270491}]

# Modelo pré-treinado no Synthetic Word Dataset + Wikipédia
## ICDAR SROIE 2019 - Avaliação no conjunto de teste

In [ ]:
trainer.test(model)


Sample Target: Company: AIK HUAT HARDWARE ENTERPRISE (SETIA ALAM) SDN BHD Address: NO. 17-G, JALAN SETIA INDAH (X) U13/X, SETIA ALAM, SEKSYEN U13, 40170 SHAH ALAM,
Prediction: Company: AIK HUAT HARDWARE ENTERPRISE (SETIA ALAM) SDN BHD Address: NO. 17-G, JALAN SETIA INDAH (X) U13/X, SETIA ALAM, SEKSYEN U13, 40170 SHAH ALAM,


Sample Target: Company: MR. D.I.Y. (M) SDN BHD Address: LOT 1851-A & 1851-B, JALAN KPB 6, KAWASAN PERINDUSTRIAN BALAKONG, 43300 SERI KEMBANGAN, SELANGOR
Prediction: Company: MR. D.I.Y. (M) SDN BHD Address: LOT 1851-A & 1851-B, JALAN KPB 6, KAWASAN PERINDUSTRIAN BALAKONG, 43300 SERI KEMBANGAN, SELANGOR


Sample Target: Company: GERBANG ALAF RESTAURANTS SDN BHD Address: LEVEL 6, BANGUNAN TH, DAMANSARA UPTOWN3 NO.3, JALAN SS21/39, 47400 PETALING JAYA SELANGOR
Prediction: Company: YHM AEON TEBRAU CITY Address: S117, SECOND FLOOR, AEON TEBRAU CITY, 1, JALAN DESA TEBRAU, TAMAN DESA TEBRAU, 81100 JOHOR BAHRU, JOHOR.


Sample Target: Company: MR. D.I.Y. (KUCHAI) SDN BHD A

[{'test_exact': 0.23115577889447236, 'test_f1': 0.4742705727864495}]